<a href="https://colab.research.google.com/github/souravraha/galaxy/blob/experimental/Lightning_Tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prerequisites

## Install/uninstall packages

In [1]:
# If you are running on Google Colab, uncomment below to install the necessary dependencies 
# before beginning the exercise.

print('Setting up colab environment')
# !pip uninstall -y -q pyarrow
!pip install -q lightning-bolts GPy
!pip install -q ray[debug] ray[default]
!pip install -U -q ray[tune]
# !pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.8-cp37-cp37m-linux_x86_64.whl

# # A hack to force the runtime to restart, needed to include the above dependencies.
# print('Done installing! Restarting via forced crash (this is not an issue).')
# import os
# os._exit(0)

Setting up colab environment
     |████████████████████████████████| 253 kB 4.1 MB/s 
     |████████████████████████████████| 959 kB 34.2 MB/s 
     |████████████████████████████████| 919 kB 18.3 MB/s 
     |████████████████████████████████| 282 kB 22.4 MB/s 
     |████████████████████████████████| 829 kB 32.0 MB/s 
     |████████████████████████████████| 119 kB 45.6 MB/s 
     |████████████████████████████████| 636 kB 52.8 MB/s 
     |████████████████████████████████| 1.3 MB 33.9 MB/s 
     |████████████████████████████████| 71 kB 8.9 MB/s 
     |████████████████████████████████| 142 kB 47.3 MB/s 
     |████████████████████████████████| 294 kB 50.6 MB/s 
     |████████████████████████████████| 49.6 MB 6.0 kB/s 
     |████████████████████████████████| 72 kB 504 kB/s 
     |████████████████████████████████| 201 kB 1.7 MB/s 
     |████████████████████████████████| 127 kB 48.3 MB/s 
     |████████████████████████████████| 65 kB 3.5 MB/s 
     |████████████████████████████████| 3.1 MB 40.5

## Import libraries

In [2]:
import os
import math
import numpy as np
from itertools import cycle
from matplotlib import pyplot as plt
# ------------------------------------
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
# ------------------------------------
from torchvision.models import resnet18
from torchvision.utils import save_image, make_grid
from torchvision import transforms, datasets
# ------------------------------------
import torchmetrics as tm
# ------------------------------------
import pytorch_lightning as pl
from pl_bolts.models.gans import DCGAN
# from Ca
from pl_bolts.callbacks import ModuleDataMonitor
from pytorch_lightning.loggers import TensorBoardLogger
from pl_bolts.models.self_supervised.resnets import BasicBlock
from pytorch_lightning.utilities.cloud_io import load as pl_load
# ------------------------------------
from ray import tune
from ray.tune.stopper import TrialPlateauStopper
from ray.tune import CLIReporter, JupyterNotebookReporter
from ray.tune.schedulers import ASHAScheduler, PopulationBasedTraining
from ray.tune.schedulers.pb2 import PB2
from ray.tune.integration.pytorch_lightning import TuneReportCallback, TuneReportCheckpointCallback
# ------------------------------------

# Download and extract data

In [45]:
from google.colab import drive
# drive.flush_and_unmount()
drive.mount('/content/drive')
# %cp drive/MyDrive/ml/Callbacks/confused_logits.py ./
# from confused_logits import ConfusedLogitCallback

Mounted at /content/drive


Here choose the model you wish to use for training/testing. Don't forget to make modifications in the following sections:

1.   GLOBAL in class definition of npyImageData.
2.   correct assignment of metric keys while defining the training wrapper for Tune.
3.   name of the experiment initiated/resumed.

In [4]:
# 'a': 1Cjcw2EWorhdhJSGoWOdxsEUDxvl943dt, 'b': 15yXXC4h5VsytP3Ak1jfUSjQhdgP2s23K, 'c': 1vuQ-pLzoKT4Hd_V7949r9eND9E2fB_u_,
# 'd': , 'e': 1wFuasvb7PthxXtMUlsD13uzYHWlWt06H, 'f': 17l6H61tLAu26zGuei38r_T5ssjbYUeaJ, 
# 'g': 1SxQVosWeEjY3Pyn8LRXA11rLnZ9HK_7B, 'h': 1Atau0RH4oyLAiYReW-G9a8l9pUNltglF, 'i': 15lEgsR1p00KSHieaT9a1nkbJ86pDxwgp, 
# 'j': 1m0EQUbqZZeyl76XsQIKWU5Qd7jGmmWhB, 'k': , 'l': 1meTDi4aeWfdChOiXeLtUOGhjVDVu000e

# !rm -rf images
!gdown --id 17l6H61tLAu26zGuei38r_T5ssjbYUeaJ
!tar zxf ./model_f.tgz
!rm ./model_f.tgz

# def prepare_data(data_dir: str = '/content'):
#     with FileLock(os.path.expanduser(data_dir+'.lock')):
#         gdown.download('https://drive.google.com/uc?id=17l6H61tLAu26zGuei38r_T5ssjbYUeaJ', data_dir+'/model_j.tgz', quiet=True)
        
#         temp = tarfile.open(data_dir+'/model_j.tgz', 'r|gz')
#         temp.extractall()
#         temp.close()

Downloading...
From: https://drive.google.com/uc?id=17l6H61tLAu26zGuei38r_T5ssjbYUeaJ
To: /content/model_f.tgz
2.34GB [00:27, 85.1MB/s]


# Class definitions

## DataModule
This creates dataloaders which need to be supplied to train, validate or test the module we have.

In [5]:
class npyImageData(pl.LightningDataModule):
    def __init__(self, config, img_width: int = 150, data_dir: str = '/content/images/'):
        super().__init__()
        # This method is not implemented
        # self.save_hyperparameters()
        self.bs = config['bs']
        self.data_dir = os.path.expanduser(data_dir)
        
        # Change the source file containing mean and stdv when changing dataset ------------------------------------------------------
        self.transform = transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip(),
            # F : [mean=71.75926373866668, std=96.139484964214, min=5.0, range=961.0]
            # J : [mean=50.271541595458984, std=94.8838882446289, min=0, range=1007.0]
            transforms.Normalize(mean=(5,), std=(961,)),
            transforms.Normalize(mean=(0.5,), std=(0.5,)),
            # this shift-scales the pixel values -> [-1, 1]
            transforms.Resize(img_width, transforms.InterpolationMode.NEAREST),
        ])

    @staticmethod
    def npy_loader(path):
        # s=np.load(path).astype('float',copy=False)
        return torch.from_numpy(np.load(path)).unsqueeze(0).float()
        # Convert to tenssor first, and then to float, otherwise final dtype 
        # would be float64, which would raise errors in conv layers      ###### type as

    def setup(self, stage: str = None):
        if stage in ('fit', None):
            self.train_set = datasets.DatasetFolder(os.path.join(self.data_dir,'train'), 
                self.npy_loader, ('.npy'), self.transform,)
            # self.train_set, self.val_set = random_split(self.full_set, [60000, 15000])            
            self.val_set = datasets.DatasetFolder(os.path.join(self.data_dir,'val'),  
                self.npy_loader, ('.npy'), self.transform,)
            self.dims = tuple(self.train_set[0][0].shape)

        if stage in ('test', None):
            self.test_set = datasets.DatasetFolder(os.path.join(self.data_dir,'val'),  
                self.npy_loader, ('.npy'), self.transform,)
            self.dims = getattr(self, 'dims', self.test_set[0][0].shape)
    
    def train_dataloader(self):
        return DataLoader(self.train_set, self.bs, shuffle=True, num_workers=os.cpu_count(), pin_memory=False)

    def val_dataloader(self):
        return DataLoader(self.val_set, self.bs, shuffle=True, num_workers=os.cpu_count(), pin_memory=False)

    def test_dataloader(self):
        return DataLoader(self.test_set, self.bs, shuffle=True, num_workers=os.cpu_count(), pin_memory=False)

## ResNet
We modify a ResNet slightly for our purpose.

In [6]:
PRE_F_RESNET = '/content/drive/MyDrive/Logs/F/LensResnet/PRETRAINED.pth'
PRE_J_RESNET = '/content/drive/MyDrive/Logs/J/LensResnet/PRETRAINED.pth'

class LensResnet(pl.LightningModule):
    def __init__(self, config, image_channels: int = 1, num_classes: int = 3, **kwargs):
        super().__init__()
        self.save_hyperparameters(ignore=config)
        self.lr = config['lr']

        # --------------------------------------------------------------------------
        self.backbone = torch.load(PRE_F_RESNET, map_location=self.device)
        
        self.train_metrics = tm.MetricCollection([tm.AUROC(self.hparams.num_classes, average='weighted'),],
            prefix='LensResnet/train/'
        )
        self.val_metrics = tm.MetricCollection([tm.AUROC(self.hparams.num_classes, average=None),
                                                tm.ROC(self.hparams.num_classes),],
        )

    def configure_optimizers(self):
        return torch.optim.Adam(self.backbone.parameters(), self.lr)

    def forward(self, x, prob=False):
        logits = self.backbone(x)
        return F.softmax(logits, 1) if prob else logits

    def training_step(self, batch, batch_idx):
        imgs, labels = batch
        self.last_logits = self(imgs)
        loss = F.cross_entropy(self.last_logits, labels)
        self.log('LensResnet/train/loss', loss)
        #  keep only scalars here, for no errors
        
        preds = F.softmax(self.last_logits, 1)
        self.train_metrics.update(preds, labels)
        try:
            self.log_dict(self.train_metrics.compute(), prog_bar=True)
        except Exception as f:
            print(f)
        finally:            
            # self.train_metrics.reset()
            # self.log_dict automatically resets at the end of epoch
            return loss

    def validation_step(self, batch, batch_idx):
        imgs, labels = batch
        logits = self(imgs)
        loss = F.cross_entropy(logits, labels)
        self.log('LensResnet/val/loss', loss)
        #  keep only scalars here, for no errors
        
        preds = F.softmax(logits, 1)
        self.val_metrics.update(preds, labels)
        # if batch_idx % 5 ==0:
        #     print('val', batch_idx, len(self.val_metrics.AUROC.target))

    def validation_epoch_end(self, Listofdicts):
        # prediction, target = torch.cat([x['pred'] for x in Listofdicts]), torch.cat([x['target'] for x in Listofdicts])
        # aurocTensor = tm.functional.auroc(prediction, target, num_classes=self.hparams.num_classes, average=None)
        try:
            scoresDict = self.val_metrics.compute()
        except Exception as f:
            # print(f)
            print(torch.unique(torch.stack(self.val_metrics.AUROC.target)).tolist())
        else:
            self.log('LensResnet/val/auroc', scoresDict['AUROC'].min(), prog_bar=True)
            fprList, tprList, _ = scoresDict['ROC']  # tm.functional.roc(prediction, target, num_classes=self.hparams.num_classes)
            
            f = plt.figure()
            colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
            for i, color in zip(range(self.hparams.num_classes), colors):
                plt.plot(fprList[i].cpu(), tprList[i].cpu(), color=color,
                        label='ROC curve of class {0} (area = {1:0.2f})'
                        ''.format(i, scoresDict['AUROC'][i]))
            plt.plot([0, 1], [0, 1], 'k--')
            plt.xlim([0.0, 1.0])
            plt.ylim([0.0, 1.05])
            plt.xlabel('False Positive Rate')
            plt.ylabel('True Positive Rate')
            plt.title('Multi-class ROC')
            plt.legend(loc='lower right')

            self.logger.experiment.add_figure('LensResnet/val/ROC', f)
            f.savefig(str(self.trainer.log_dir) + '/ROC_epoch_{:02d}.pdf'.format(self.current_epoch))
        finally:
            self.val_metrics.reset()

## Stage 1
Here we subclass a DCGAN to create our low resolution GAN.

In [ ]:
m = Stage1({'lr':0.001, 'n_fmaps': 128, 'bs': 8})

In [46]:
BEST_F_RESNET = '/content/drive/MyDrive/Logs/F/LensResnet/pbt_tanh/train_LensResnet_eb619_00000_0_2021-09-02_19-42-34/checkpoint_epoch=2-step=28124'
BEST_J_RESNET = '/content/drive/MyDrive/Logs/J/LensResnet/pbt_tanh_fine/train_LensResnet_93609_00000_0_2021-09-02_21-06-00/checkpoint_epoch=2-step=28124'

def post_plotting(ax):
    ax.plot([0, 1], [0, 1], 'k--')
    ax.legend(loc='lower right')

class Stage1(DCGAN):
    def __init__(self, config, num_classes: int = 3, **kwargs):
        super().__init__(feature_maps_gen=config['n_fmaps'], feature_maps_disc=config['n_fmaps'], learning_rate=config['lr'])
        self.save_hyperparameters(ignore=config)

        self.generator.add_module('emb', nn.Embedding(self.hparams.num_classes, self.hparams.latent_dim))
        
        del self.discriminator.disc[-1][-1]
        self.criterion = nn.BCEWithLogitsLoss()

        lensF = LensResnet.load_from_checkpoint(os.path.join(BEST_F_RESNET, 'checkpoint')).eval()
        self.modelF = lensF.backbone
        # self.modelF = torch.load(PRE_F_RESNET, map_location=self.device).eval()
        self.modelF.fc = nn.Identity()
        self.lastF = lensF.backbone.fc
        
        lensJ = LensResnet.load_from_checkpoint(os.path.join(BEST_J_RESNET, 'checkpoint')).eval()
        self.modelJ = lensJ.backbone
        # self.modelJ = torch.load(PRE_J_RESNET, map_location=self.device).eval()
        self.modelJ.fc = nn.Identity()
        self.lastJ = lensJ.backbone.fc

        self.val_metrics = tm.MetricCollection(
            {
                'FID_F' : tm.FID(self.modelF),
                'FID_J' : tm.FID(self.modelJ),
            },
            prefix='Stage1/val/',
        )

    def forward(self, noise, labels):
        inp = noise.mul(self.generator.emb(labels))
        return super().forward(inp)

    def training_step(self, batch, batch_idx, optimizer_idx):
        real, labels = batch
        fake = self._get_fake_pred(labels).type_as(real)

        # Train discriminator
        result = None
        if optimizer_idx == 0:
            result = self._disc_step(real, fake.detach())

        # Train generator
        if optimizer_idx == 1:
            result = self._gen_step(fake)

        return result

    def _disc_step(self, real, fake):
        disc_loss = self._get_disc_loss(real, fake)
        self.log('Stage1/D/train/loss', disc_loss)
        return disc_loss

    def _gen_step(self, fake):
        gen_loss = self._get_gen_loss(fake)
        self.log('Stage1/G/train/loss', gen_loss)
        return gen_loss

    def _get_disc_loss(self, real, fake):
        # Train with real
        real_pred = self.discriminator(real)
        real_gt = torch.ones_like(real_pred)
        real_loss = self.criterion(real_pred, real_gt)

        # Train with fake
        fake_pred = self.discriminator(fake)
        fake_gt = torch.zeros_like(fake_pred)
        fake_loss = self.criterion(fake_pred, fake_gt)

        disc_loss = real_loss + fake_loss

        return disc_loss

    def _get_gen_loss(self, fake):
        # Train with fake
        fake_pred = self.discriminator(fake)
        fake_gt = torch.ones_like(fake_pred)
        gen_loss = self.criterion(fake_pred, fake_gt)

        return gen_loss

    def _get_fake_pred(self, labels):
        batch_size = len(labels)
        noise = self._get_noise(batch_size, self.hparams.latent_dim)
        fake = self(noise, labels)
        # fake_pred = self.discriminator(fake)

        return fake

    def validation_step(self, batch, batch_idx):
        imgs64, labels = batch
        self.val_metrics.update(F.interpolate(imgs64, 150), real=True)

        fake = F.interpolate(self._get_fake_pred(labels), 150).type_as(imgs64)
        self.val_metrics.update(fake, real=False)
        return {'predF': F.softmax(self.lastF(self.modelF(fake)), 1), 
                'predJ': F.softmax(self.lastJ(self.modelJ(fake)), 1), 
                'target': labels}

    def validation_epoch_end(self, ListofDicts):
        self.log_dict(self.val_metrics)
        
        target = torch.cat([x['target'] for x in ListofDicts])
        fig, ax = plt.subplots(1,2, 
            subplot_kw={'xlim': [0,1], 'xlabel': 'False Positive Rate', 'ylim': [0,1.05], 
                        'ylabel': 'True Positive Rate',
            },
            figsize=[11, 5],
        )
        for j, letter in enumerate(['F', 'J']):
            prediction = torch.cat([x['pred' + letter] for x in ListofDicts])
            aurocTensor = tm.functional.auroc(prediction, target, num_classes=self.hparams.num_classes, average=None)
            self.log('Stage1/LensResnet(' + letter + ')/val/auroc', aurocTensor.min())
            fprList, tprList, _ = tm.functional.roc(prediction, target, num_classes=self.hparams.num_classes)
            
            colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
            for i, color in zip(range(self.hparams.num_classes), colors):
                ax[j].plot(fprList[i].cpu(), tprList[i].cpu(), color=color,
                        label='ROC curve of class {0} (area = {1:0.2f})'
                        ''.format(i, aurocTensor[i]))
            post_plotting(ax[j])
            ax[j].set_title('Multi-class ROC (' + letter + ')')
        
        fig.tight_layout()
        self.logger.experiment.add_figure('Stage1/LensResnet/val/ROC', fig)
        fig.savefig(str(self.trainer.log_dir) + '/ROC_epoch_{:02d}.pdf'.format(self.current_epoch))

        labels = torch.arange(self.hparams.num_classes, device=self.device)
        fake = F.interpolate(self._get_fake_pred(labels), 150)
        save_image(fake, 
                   str(self.trainer.log_dir) + '/Fake_epoch_{:02d}.pdf'.format(self.current_epoch), 
                  #  kwargs for make_grid
                   normalize=True, value_range=(-1,1))

In [ ]:
features=[]
def hook(self, _, output):
    features.append(output)
handles = [i.register_forward_hook(hook) for i in m.generator.gen]
labels = torch.arange(m.hparams.num_classes, device=m.device)
_ = m._get_fake_pred(labels)

In [ ]:
[i.remove() for i in handles]
del handles

In [ ]:
for i, tensor in enumerate(features):
    shape = tensor.shape
    print(math.ceil(math.sqrt(shape[1]))*shape[2])
    save_image(tensor, 
                'features_{:02d}.png'.format(i), 
                #  kwargs for make_grid
                normalize=True, value_range=(-1,1), nrow=3)
    # [i.remove() for i in handles]

## LensGAN128
Here we extend the Stage1 gan to create 128 by 128 pixel images.

In [35]:
class LensGAN128(Stage1):
    def __init__(self, config, num_classes: int = 3):
        super().__init__(config, num_classes=num_classes)
        
        self.generator.gen[0] = self.generator._make_gen_block(self.hparams.feature_maps_gen * 16, self.hparams.feature_maps_gen * 8)
        first = self.generator._make_gen_block(self.hparams.latent_dim, self.hparams.feature_maps_gen * 16, kernel_size=4, stride=1, padding=0)
        self.generator.gen = nn.Sequential(first, *list(self.generator.gen))

        self.discriminator.disc[-1] = self.discriminator._make_disc_block(self.hparams.feature_maps_disc * 8, self.hparams.feature_maps_disc * 16)
        last = self.discriminator._make_disc_block(self.hparams.feature_maps_disc * 16, 1, kernel_size=4, stride=1, padding=0, last_block=True)[0]
        self.discriminator.disc = nn.Sequential(*list(self.discriminator.disc), last)

In [32]:
m = Stage1({'lr': 1e-4, 'n_fmaps':7, 'bs': 8}, 4)

 /usr/local/lib/python3.7/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning:Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
 /usr/local/lib/python3.7/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning:Metric `ROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
 /usr/local/lib/python3.7/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning:Metric `FID` will save all extracted features in buffer. For large datasets this may lead to large memory footprint.


## Stage 2
Here we subclass a DCGAN to create our high resolution GAN.

In [ ]:
class Generator2(nn.Module):
    def __init__(self, ngf: int = 128, image_channels: int = 1, res_depth: int = 6):
        super().__init__()

        ker, strd = 4, 2
        pad = int((ker - 2)/2)
        res_ker, res_strd, res_pad = 3, 1, 1
        
        # 64 -> 32
        self.preprocessing = nn.Sequential(
            nn.Conv2d(image_channels, ngf, ker, strd, pad, bias=False),
            nn.ReLU(True)
        )
        # residuals
        layer = []
        for _ in range(res_depth):
            layer.append(BasicBlock(ngf, ngf))
        self.residual = nn.Sequential(*layer)
        
        self.ending_residual = nn.Sequential(
            nn.Conv2d(ngf, ngf, res_ker, res_strd, res_pad, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True)
        )

        # at this part, add the residual inputs from after the preprocessing

        image_width = 150 # upscaling should be factor of 2 increase
        mode = 'nearest' # upscaling method is nearest-neighbour
        self.main = nn.Sequential(
            # 32 -> 75
            nn.Upsample(image_width//2, mode=mode),
            nn.Conv2d(ngf, ngf*4, res_ker, res_strd, res_pad, bias=False),
            nn.BatchNorm2d(ngf*4),
            nn.ReLU(True),
            # 75 -> 150
            nn.Upsample(image_width, mode=mode),
            nn.Conv2d(ngf*4, image_channels, res_ker, res_strd, res_pad, bias=False),
            nn.Tanh()
        )

    def forward(self, in_x):
        x_p = self.preprocessing(in_x)
        x_r = x_p
        x_r = self.residual(x_r)
        x_r = self.ending_residual(x_r)
        # large residual connections
        x_f = x_r + x_p
        return self.main(x_f)

In [ ]:
BEST_F_STAGE1 = '/content/drive/MyDrive/Logs/F/Stage1/pbt_tanh_1/train_Stage1_90727_00001_1_n_fmaps=16_2021-08-30_08-02-05/checkpoint_epoch=4-step=1988/'
BEST_J_STAGE1 ='/content/drive/MyDrive/Logs/J/Stage1/pbt_tanh/train_Stage1_28c03_00003_3_n_fmaps=64_2021-08-31_20-45-44/checkpoint_epoch=2-step=1403/'

In [ ]:
class Stage2(DCGAN):
    def __init__(self, config, num_classes: int = 3, **kwargs):
        super().__init__(feature_maps_gen=config['n_fmaps'], feature_maps_disc=config['n_fmaps'], learning_rate=config['learning_rate'])
        self.save_hyperparameters(ignore=config)

        self.generator = Generator2(self.hparams.feature_maps_gen, self.hparams.image_channels, config['res_depth'])

        # These are better as attributes, instead of being returned by a method
        self.modelF = getattr(self, 'modelF', LensResnet.load_from_checkpoint(os.path.join(BEST_RESNET_F, 'checkpoint')).eval())
        self.modelJ = getattr(self, 'modelJ', LensResnet.load_from_checkpoint(os.path.join(BEST_RESNET_J, 'checkpoint')).eval())
        # Workaround:
        self.lowres = getattr(self, 'lowres', Stage1.load_from_checkpoint(os.path.join(BEST_STAGE1_F, 'checkpoint')).eval())
        
        metrics = tm.MetricCollection(
            [
             tm.AUROC(num_classes=self.hparams.num_classes, compute_on_step=False, average=None), 
             tm.ROC(num_classes=self.hparams.num_classes, compute_on_step=False),
            ]
        )
        self.metricsF = metrics.clone()
        self.metricsJ = metrics.clone()

    def forward(self, noise):
        return self.generator(noise)

    def training_step(self, batch, batch_idx, optimizer_idx):
        real, self.labels = batch

        # Train discriminator
        result = None
        if optimizer_idx == 0:
            result = self._disc_step(real)

        # Train generator
        if optimizer_idx == 1:
            result = self._gen_step(real)

        return result

    def _disc_step(self, real):
        disc_loss = self._get_disc_loss(real)
        self.log('Stage2/D/train/loss', disc_loss, on_epoch=True)
        return disc_loss

    def _gen_step(self, real):
        gen_loss = self._get_gen_loss(real)
        self.log('Stage2/G/train/loss', gen_loss, on_epoch=True)
        return gen_loss

    def _get_gen_loss(self, real: torch.Tensor) -> torch.Tensor:
        # Train with fake
        fake_pred = self._get_fake_pred(real)
        fake_gt = torch.ones_like(fake_pred)
        gen_loss = self.criterion(fake_pred, fake_gt)

        # class_pred =  self._get_class_pred(len(real))
        # gen_loss += F.cross_entropy(class_pred, self.labels)

        return gen_loss

    def _get_class_pred(self, batch_size) -> torch.Tensor:
        # ----------------------------------------------------------------------------------------------------------------
        return self.modelF.backbone(self(self._get_noise(batch_size, self.hparams.latent_dim)))

    def _get_noise(self, n_samples: int, latent_dim: int, labels = None):
        # can't use self in function definition
        if labels is None:
            labels = self.labels
            # getattr(self, 'labels', torch.randint(self.hparams.num_classes, (n_samples,), device=self.device))  # last dimension is the hidden dimension
        return self.lowres(super()._get_noise(n_samples, latent_dim), labels)

    def validation_step(self, batch, batch_idx):
        imgs, labels = batch
        out = self(self._get_noise(labels.shape[0], self.hparams.latent_dim, labels))
        self.metricsF.update(self.modelF(out), labels)
        self.metricsJ.update(self.modelJ(out), labels)
        # out = Fig.interpolate(out_64, 150)

    def validation_epoch_end(self, listofDicts):
        fig, ax = plt.subplots(1,2, 
            subplot_kw={'xlim': [0,1], 'xlabel': 'False Positive Rate', 'ylim': [0,1.05], 
                        'ylabel': 'True Positive Rate',
            },
            figsize=[11, 5],
        )
        for j, letter in enumerate(['F', 'J']):
            output = getattr(self, 'metrics' + letter).compute()
            self.log('Stage2/ResNet(' + letter + ')/val/auroc', output['AUROC'].min())
            fprList, tprList, _ = output['ROC']
            
            colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
            for i, color in zip(range(self.hparams.num_classes), colors):
                ax[j].plot(fprList[i].cpu(), tprList[i].cpu(), color=color,
                        label='ROC curve of class {0} (area = {1:0.2f})'
                        ''.format(i, output['AUROC'][i]))
            post_plotting(ax[j])
            ax[j].set_title('Multi-class ROC (' + letter + ')')
        
        fig.tight_layout()
        self.logger.experiment.add_figure('Stage2/ResNet/val/ROC', fig)
        fig.savefig(str(self.trainer.log_dir) + '/ROC_epoch_{:02d}.pdf'.format(self.current_epoch))

        labels = torch.arange(self.hparams.num_classes, device=self.device)
        save_image(self(self._get_noise(labels.shape[0], self.hparams.latent_dim, labels)), 
                   str(self.trainer.log_dir) + '/Fake_epoch_{:02d}.pdf'.format(self.current_epoch), 
                  #  kwargs for make_grid
                   normalize=True, value_range=(-1,1))

    def on_fit_end(self):
        delattr(self, 'modelF')
        delattr(self, 'modelJ')
        delattr(self, 'labels')
        delattr(self, 'lowres')

## StackGAN:
Here we define the full GAN module, that we shall use to generate representative images.

In [ ]:
class StackGAN(pl.LightningModule):
    def __init__(self, config, noise_size: int = 100, image_width = 64,
                    num_classes: int = 3, image_channels: int = 1, b1: float = 0.5, **kwargs):
        super().__init__()
        self.save_hyperparameters(ignore = config)
        self.feature_maps = config['feature_maps']
        self.lr = config['learning_rate']
        # -------------------------------------
        # Need to create a subclass because we couldn't simply add/remove a layer;
        # there are two inputs of the superclas' forward method.
        self.G1 = DCGANGenerator(self.hparams.noise_size, self.feature_maps, self.hparams.image_channels).apply(self._weights_init)
        l = list(self.G1.gen[0])
        del l[1]
        self.G1.gen[0] = nn.Sequential(*l)
        self.G1.add_module('label_emb', nn.Embedding(self.hparams.num_classes, self.hparams.noise_size))
        # ------------------------------------
        self.D1 = DCGANDiscriminator(self.feature_maps, self.hparams.image_channels).apply(self._weights_init)
        # -------------------------------------
        self.G2 = Generator2(self.hparams.image_channels, self.feature_maps).apply(self._weights_init)
        # -------------------------------------
        self.D2 = DCGANDiscriminator(self.feature_maps, self.hparams.image_channels)
        #  steps to mutate the instance, not the class definition
        extra = self.D2._make_disc_block(self.feature_maps * 2, self.feature_maps * 2)
        l = list(self.D2.disc)
        l.insert(2, extra)
        self.D2.disc = nn.Sequential(*l)
        self.D2.apply(self._weights_init)
        # No need for subclassing as the forward method need not be modified.
        # -------------------------------------
        self.R = LensResnet(config, num_classes = 4).apply(self._weights_init)
        # -------------------------------------
        self.pretrained = LensResnet(config)
        ckpt = pl_load(os.path.join(
            '/content/drive/MyDrive/Logs/tune_LensResnet_asha_model_j/train_LensResnet_tune_checkpoint_e38cb_00000_0_batch_size=128,learning_rate=0.001_2021-07-06_17-52-11/checkpoint_epoch=17-step=1406',
            # '/content/drive/MyDrive/Logs/tune_LensResnet_asha_model_f/train_LensResnet_tune_checkpoint_e32ba_00000_0_batch_size=64,learning_rate=0.0001_2021-07-06_03-33-10/checkpoint_epoch=14-step=4689',
            'checkpoint'),
            map_location=lambda storage, loc: storage)
        self.pretrained._load_model_state(ckpt)
        # -------------------------------------
        self.criterion1 = nn.BCELoss()
        self.criterion2 = nn.CrossEntropyLoss()

    @staticmethod
    def _weights_init(m):
        classname = m.__class__.__name__
        if classname.find('Conv') != -1:
            torch.nn.init.normal_(m.weight, 0.0, 0.02)
        elif classname.find('BatchNorm') != -1:
            torch.nn.init.normal_(m.weight, 1.0, 0.02)
            torch.nn.init.zeros_(m.bias)

    def forward(self, noise, labels = None):
        if labels is None:
            labels = torch.randint(self.hparams.num_classes, noise.shape[:-1])                           # last dimension is the hidden dimension
        inp = torch.mul(noise, self.G1.label_emb(labels))
        out1 = self.G1(inp.view(-1, inp.shape[-1], 1, 1))
        out2 = self.G2(out1.detach())
        return out2, out1

    def training_step(self, batch, batch_idx, optimizer_idx):
        imgs, labels = batch
        temp2, temp1 = self(torch.randn(labels.shape[0], self.hparams.noise_size).type_as(imgs), labels)

        if optimizer_idx == 0:
            loss = self.criterion1(self.D1(temp1), torch.ones_like(labels, dtype=torch.float32))
            self.log('G1/train/loss/disc', loss)
            loss.add_(self.criterion2(self.R.backbone(self.G2(temp1)), labels))
            self.log('G1/train/loss/full', loss)

        elif optimizer_idx == 1:
            real, fake = self.D1(F.interpolate(imgs, self.hparams.image_width, mode='nearest')), self.D1(temp1.detach())
            prediction, target = torch.cat((real, fake)), torch.cat((torch.ones_like(real),torch.zeros_like(fake)))
            loss = self.criterion1(prediction, target)
            self.log('D1/train/loss', loss)

        elif optimizer_idx == 2:
            loss = self.criterion1(self.D2(temp2), torch.ones_like(labels, dtype=torch.float32))
            self.log('G2/train/loss/disc', loss)
            loss.add_(self.criterion2(self.R.backbone(temp2), labels))
            self.log('G2/train/loss/full', loss)

        elif optimizer_idx == 3:
            real, fake = self.D2(imgs), self.D2(temp2.detach())
            prediction, target = torch.cat((real, fake)), torch.cat((torch.ones_like(real),torch.zeros_like(fake)))
            loss = self.criterion1(prediction, target)
            self.log('D2/train/loss', loss)

        elif optimizer_idx == 4:
            real, fake = self.R.backbone(imgs), self.R.backbone(temp2.detach())
            prediction, target = torch.cat((real, fake)), torch.cat((labels, self.hparams.num_classes * torch.ones_like(labels)))
            loss = self.criterion2(prediction, target)
            self.log('R/train/loss', loss)
        
        return loss

    def configure_optimizers(self):
        opt_g1 = torch.optim.Adam(self.G1.parameters(), self.lr, (self.hparams.b1, 0.999))
        opt_d1 = torch.optim.Adam(self.D1.parameters(), self.lr, (self.hparams.b1, 0.999))
        opt_g2 = torch.optim.Adam(self.G2.parameters(), self.lr, (self.hparams.b1, 0.999))
        opt_d2 = torch.optim.Adam(self.D2.parameters(), self.lr, (self.hparams.b1, 0.999))
        opt_r = torch.optim.Adam(self.R.parameters(), self.lr, (self.hparams.b1, 0.999))
        return opt_g1, opt_d1, opt_g2, opt_d2, opt_r

    def validation_step(self, batch, batch_idx):
        imgs, labels = batch
        temp2, _ = self(torch.randn(labels.shape[0], self.hparams.noise_size).type_as(imgs), labels)
        return {'pred': self.pretrained(temp2.detach()), 'target': labels}

    def validation_epoch_end(self, listofDicts):
        prediction, target = torch.cat([x['pred'] for x in listofDicts]), torch.cat([x['target'] for x in listofDicts])
        aurocTensor = tm.functional.auroc(prediction, target, num_classes=self.hparams.num_classes, average=None)
        self.log('Pre/val/auroc', aurocTensor.min())
        fprList, tprList, _ = tm.functional.roc(prediction, target, num_classes=self.hparams.num_classes)
        
        f = plt.figure()
        colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
        for i, color in zip(range(self.hparams.num_classes), colors):
            plt.plot(fprList[i].cpu(), tprList[i].cpu(), color=color,
                    label='ROC curve of class {0} (area = {1:0.2f})'
                    ''.format(i, aurocTensor[i].cpu()))
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Multi-class ROC')
        plt.legend(loc='lower right')

        self.logger.experiment.add_figure('StackGAN/val/ROC', f)
        f.savefig(str(tune.get_trial_dir())+'ROC_epoch_'+str(self.current_epoch)+'.pdf')

# Tune Hyperparameters


## ResNet
Here we tune hyperparameters as we train our modified ResNet.

In [ ]:
%rm -rf ./drive/MyDrive/Logs/F/LensResnet/pbt_tanh

In [ ]:
# __tune_train_checkpoint_begin
def train_LensResnet(config, checkpoint_dir=None, num_epochs=10, num_gpus=torch.cuda.device_count()):
    # print(os.cpu_count(), torch.cuda.device_count())
    kwargs = {
        # 'limit_train_batches' : 0.005,
        # 'limit_val_batches' : 0.005,
        'progress_bar_refresh_rate' : math.ceil(8250//config['bs']),
        'max_epochs' : num_epochs,
        'prepare_data_per_node' : False,
        # If fractional GPUs passed in, convert to int.
        'gpus' : math.ceil(num_gpus),
        'logger' : TensorBoardLogger(save_dir=tune.get_trial_dir(), name='', version='.'),
        'callbacks' : [
            TuneReportCheckpointCallback(
                {
                    'loss': 'LensResnet/val/loss', 
                    'auroc': 'LensResnet/val/auroc', 
                },
            ),
            ModuleDataMonitor(['backbone.layer2', 'backbone.layer4', 'backbone.fc']),
            ConfusedLogitCallback(5),
        ],
        'stochastic_weight_avg' : True,
        # works with only one optimizer
        'benchmark' : True,
        'precision' : 16,     # can't use on cpu
        # 'track_grad_norm': 2,
        # 'gradient_clip_val' : 0.5, 
        # 'gradient_clip_algorithm' : 'value',
    }
    
    dm = npyImageData(config)                                              # Specify image width here    
    if checkpoint_dir is not None:
        kwargs['resume_from_checkpoint'] = os.path.join(checkpoint_dir, 'checkpoint')
        # model = LensResnet.load_from_checkpoint(kwargs['resume_from_checkpoint'], config=config)
    # else:

    model = LensResnet(config)
    trainer = pl.Trainer(**kwargs)

    trainer.fit(model, dm)
# __tune_train_checkpoint_end__

# __tune_pbt_begin__
def tune_LensResnet_pbt(num_samples=10, num_epochs=10, gpus_per_trial=torch.cuda.device_count()):
    # print(os.cpu_count(), torch.cuda.device_count())
    analysis = tune.run(
        tune.with_parameters(
            train_LensResnet,
            num_epochs=num_epochs,
            num_gpus=gpus_per_trial
        ),
        # Change the folder name when changing dataset--------------------------------------------------------------------------
        name='J/LensResnet/pbt_tanh_fine',
        metric='loss',
        mode='min',
        # stop=TrialPlateauStopper('auroc'),
        resources_per_trial={'cpu': os.cpu_count(), 'gpu': gpus_per_trial},
        local_dir='./drive/MyDrive/Logs',
        # config={'lr': tune.choice([1e-4, 1e-3, 1e-5, 1e-2, 1e-6, 1e-1, 1e-7]),
        #         'bs': tune.grid_search([8, 16, 32, 64, 128]),
        #         },
        # scheduler = pbtScheduler(max_t=num_epochs, grace_period=2, reduction_factor=2),
        # Can't use RB2 as it requires mutations to be continuous
        config={'lr': 1e-5,
                'bs': 8,
                # RuntimeError: stack expects each tensor to be equal size, but got [128] at entry 0 and [120] at entry 585
                },
        scheduler = PopulationBasedTraining(time_attr='training_iteration', quantile_fraction=0.4,
                                            resample_probability=0.2,  perturbation_interval=1,
                                            hyperparam_mutations={
                                                'lr': tune.loguniform(1e-6, 1e-4),
                                                'bs': [8, 16, 32, 64, 128],
                                            },
        ),
        progress_reporter=JupyterNotebookReporter(
            overwrite=False,
            parameter_columns=['lr', 'bs'],
            metric_columns=['loss', 'auroc', 'training_iteration'],
        ),
        fail_fast = True,
        # reuse_actors=True,
        num_samples=num_samples,
        # resume='PROMPT',
    )
    BEST_J_RESNET = analysis.best_checkpoint
    print('Best checkpoint path found is: ', BEST_J_RESNET)

# __tune_pbt_end__

if __name__ == '__main__':
    import argparse

    parser = argparse.ArgumentParser()
    parser.add_argument(
        '--smoke-test', action='store_true', help='Finish quickly for testing')
    args, _ = parser.parse_known_args()

    if args.smoke_test:
        tune_LensResnet_pbt(num_samples=1, num_epochs=6, gpus_per_trial=torch.cuda.device_count())
    else:
        # pbt scheduler
        tune_LensResnet_pbt(num_samples=1, num_epochs=5, gpus_per_trial=torch.cuda.device_count())

Trial name,status,loc,lr,bs
train_LensResnet_93609_00000,PENDING,,1e-05,8


(pid=2129) /usr/local/lib/python3.7/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
(pid=2129)   warnings.warn(*args, **kwargs)
(pid=2129) /usr/local/lib/python3.7/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `ROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
(pid=2129)   warnings.warn(*args, **kwargs)
(pid=2129) Using native 16bit precision.
(pid=2129) GPU available: True, used: True
(pid=2129) TPU available: False, using: 0 TPU cores
(pid=2129) IPU available: False, using: 0 IPUs
(pid=2129) LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
(pid=2129) 
(pid=2129)   | Name          | Type             | Params
(pid=2129) ---------------------------------------------------
(pid=2129) 0 | backbone      | ResNet           | 11.2 M
(pid=2129) 1 | train_metrics | MetricColl

Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]


(pid=2129) /usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
(pid=2129)   return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
(pid=2129) /usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/deprecated_api.py:26: LightningDeprecationWarning: `Trainer.running_sanity_check` has been renamed to `Trainer.sanity_checking` and will be removed in v1.5.
(pid=2129)   "`Trainer.running_sanity_check` has been renamed to `Trainer.sanity_checking` and will be removed in v1.5."


Epoch 0: 100%|█████████▉| 10310/10313 [16:28<00:00, 10.43it/s, loss=0.635, v_num=., LensResnet/train/AUROC=0.822]
Validating: 0it [00:00, ?it/s]
Epoch 0: 100%|█████████▉| 10310/10313 [16:48<00:00, 10.22it/s, loss=0.635, v_num=., LensResnet/train/AUROC=0.822]
(pid=2129) 
Epoch 0: 100%|██████████| 10313/10313 [17:06<00:00, 10.04it/s, loss=0.635, v_num=., LensResnet/train/AUROC=0.822]


...

Result for train_LensResnet_93609_00000:
  auroc: 0.9561334848403931
  date: 2021-09-02_21-47-49
  done: false
  experiment_id: 657e38bdbb97442d81533414ecfad8f8
  hostname: 77ab158a08cd
  iterations_since_restore: 2
  loss: 0.3701208233833313
  node_ip: 172.28.0.2
  pid: 2129
  should_checkpoint: true
  time_since_restore: 2498.2101757526398
  time_this_iter_s: 1449.7577471733093
  time_total_s: 2498.2101757526398
  timestamp: 1630619269
  timesteps_since_restore: 0
  training_iteration: 2
  trial_id: '93609_00000'
  


Trial name,status,loc,lr,bs,loss,auroc,training_iteration
train_LensResnet_93609_00000,RUNNING,172.28.0.2:2129,1e-05,8,0.370121,0.956133,2


Epoch 1: 100%|██████████| 10313/10313 [24:08<00:00,  7.12it/s, loss=0.396, v_num=., LensResnet/train/AUROC=0.883, LensResnet/val/auroc=0.956]
                                                             
Epoch 2:   0%|          | 0/10313 [00:00<00:04, 2420.26it/s, loss=0.396, v_num=., LensResnet/train/AUROC=0.883, LensResnet/val/auroc=0.956]


2021-09-02 21:50:39,623	WARNING tune.py:519 -- SIGINT received (e.g. via Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C one more time (or send SIGINT/SIGKILL/SIGTERM) to skip. 


Epoch 2: 100%|█████████▉| 10310/10313 [30:14<00:00,  5.68it/s, loss=0.36, v_num=., LensResnet/train/AUROC=0.913, LensResnet/val/auroc=0.956]
Validating: 0it [00:00, ?it/s]
Validating:   0%|          | 0/938 [00:00<?, ?it/s]


In [ ]:
!cat /content/drive/MyDrive/Logs/F/LensResnet/pbt_tanh_finetune/train_LensResnet_ed4b1_00000_0_2021-08-30_02-56-52/error.txt

## Stage 1
Here we tune hyperparameters as we train our modified DCGAN.

In [11]:
%rm -rf ./drive/MyDrive/Logs/F/Stage1/pbt_tanh/

In [12]:
# __tune_train_checkpoint_begin
def train_Stage1(config, checkpoint_dir=None, num_epochs=10, num_gpus=torch.cuda.device_count()):
    # print(os.cpu_count(), torch.cuda.device_count())
    kwargs = {
        'limit_train_batches' : 0.1,
        'limit_val_batches' : 0.1,
        'progress_bar_refresh_rate' : math.ceil(8250*0.1//config['bs']),
        'max_epochs' : num_epochs,
        'prepare_data_per_node' : False,
        # If fractional GPUs passed in, convert to int.
        'gpus' : math.ceil(num_gpus),
        'logger' : TensorBoardLogger(save_dir=tune.get_trial_dir(), name='', version='.'),
        'callbacks' : [
            TuneReportCheckpointCallback(
                {
                    'loss_G': 'Stage1/G/train/loss', 
                    'loss_D': 'Stage1/D/train/loss', 
                    # Switch up the FID vlues when training on different dataset -----------------------------------------------
                    'FID': 'Stage1/val/FID_F', 
                    'FID_cross': 'Stage1/val/FID_J',
                    'auroc': 'Stage1/LensResnet(F)/val/auroc',
                    'auroc_cross': 'Stage1/LensResnet(J)/val/auroc',
                },
            ),
            ModuleDataMonitor(True),
        ],
        # 'stochastic_weight_avg' : True,
        # works with only one optimizer
        'benchmark' : True,
        'precision' : 16,
        # 'gradient_clip_val' : 0.5, 
        # 'gradient_clip_algorithm' : 'value',
    }
    
    dm = npyImageData(config, 64)                                              # Specify image width here    
    if checkpoint_dir is not None:
        kwargs['resume_from_checkpoint'] = os.path.join(checkpoint_dir, 'checkpoint')
        # model = Stage1.load_from_checkpoint(kwargs['resume_from_checkpoint'], config=config)
    # else:

    model = Stage1(config)
    trainer = pl.Trainer(**kwargs)

    trainer.fit(model, dm)
# __tune_train_checkpoint_end__

# __tune_pbt_begin__
def tune_Stage1_pbt(num_samples=10, num_epochs=10, gpus_per_trial=torch.cuda.device_count()):
    # print(os.cpu_count(), torch.cuda.device_count())
    analysis = tune.run(
        tune.with_parameters(
            train_Stage1,
            num_epochs=num_epochs,
            num_gpus=gpus_per_trial
        ),
        # Change the folder name when changing dataset--------------------------------------------------------------------------
        name='F/Stage1/pbt_tanh',
        metric='auroc',
        mode='max',
        # stop=TrialPlateauStopper('FID'),
        resources_per_trial={'cpu': os.cpu_count(), 'gpu': gpus_per_trial},
        local_dir='./drive/MyDrive/Logs',
        # config={'lr': tune.choice([1e-4, 1e-3, 1e-5, 1e-2, 1e-6, 1e-1, 1e-7]),
        #         'bs': tune.grid_search([8, 16, 32, 64, 128]),
        #         },
        # scheduler = pbtScheduler(max_t=num_epochs, grace_period=2, reduction_factor=2),
        # Can't use RB2 as it requires mutations to be continuous
        config={'lr': 1e-4,
                'n_fmaps': tune.grid_search([8, 16, 32, 64]),
                'bs': 8,
                },
        # config = {'lr': 2.340983544823817e-05, 'n_fmaps': 32, 'bs': 8},
        scheduler = PopulationBasedTraining(time_attr='training_iteration', quantile_fraction=0.25,
                                            resample_probability=0.25,  perturbation_interval=1,
                                            hyperparam_mutations={
                                                'lr': tune.loguniform(1e-7, 1e-1),
                                                'bs': [8, 16, 32, 64, 128],
                                            },
        ),
        progress_reporter=JupyterNotebookReporter(
            overwrite=False,
            parameter_columns=['lr', 'n_fmaps', 'bs'],
            metric_columns=['loss_G', 'loss_D', 'FID', 'auroc', 'FID_cross', 
                            'auroc_cross', 'training_iteration'],
        ),
        fail_fast = True,
        # reuse_actors=True,
        num_samples=num_samples,
        # resume='PROMPT',
        # restore=BEST_J_STAGE1,
    )
    # ---------------------------------------------------------------------------------------------
    BEST_F_STAGE1 = analysis.best_checkpoint
    print('Best checkpoint path found is: ', analysis.best_checkpoint)

# __tune_pbt_end__

if __name__ == '__main__':
    import argparse

    parser = argparse.ArgumentParser()
    parser.add_argument(
        '--smoke-test', action='store_true', help='Finish quickly for testing')
    args, _ = parser.parse_known_args()

    if args.smoke_test:
        tune_Stage1_pbt(num_samples=1, num_epochs=6, gpus_per_trial=torch.cuda.device_count())
    else:
        # pbt scheduler
        tune_Stage1_pbt(num_samples=1, num_epochs=10, gpus_per_trial=torch.cuda.device_count())

Trial name,status,loc,lr,n_fmaps,bs
train_Stage1_70527_00000,RUNNING,,0.0001,8,8
train_Stage1_70527_00001,PENDING,,0.0001,16,8
train_Stage1_70527_00002,PENDING,,0.0001,32,8
train_Stage1_70527_00003,PENDING,,0.0001,64,8


(pid=787) /usr/local/lib/python3.7/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
(pid=787)   warnings.warn(*args, **kwargs)
(pid=787) /usr/local/lib/python3.7/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `ROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
(pid=787)   warnings.warn(*args, **kwargs)
(pid=787) /usr/local/lib/python3.7/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `FID` will save all extracted features in buffer. For large datasets this may lead to large memory footprint.
(pid=787)   warnings.warn(*args, **kwargs)
(pid=787) Using native 16bit precision.
(pid=787) GPU available: True, used: True
(pid=787) TPU available: False, using: 0 TPU cores
(pid=787) IPU available: False, using: 0 IPUs
(pid=787) LOCAL_RANK: 0 - CUDA_V

Trial name,status,loc,lr,n_fmaps,bs
train_Stage1_70527_00000,RUNNING,,0.0001,8,8
train_Stage1_70527_00001,PENDING,,0.0001,16,8
train_Stage1_70527_00002,PENDING,,0.0001,32,8
train_Stage1_70527_00003,PENDING,,0.0001,64,8


(pid=787) 
(pid=787)   | Name          | Type               | Params
(pid=787) -----------------------------------------------------
(pid=787) 0 | generator     | DCGANGenerator     | 146 K 
(pid=787) 1 | discriminator | DCGANDiscriminator | 44.4 K
(pid=787) 2 | criterion     | BCEWithLogitsLoss  | 0     
(pid=787) 3 | modelF        | ResNet             | 11.2 M
(pid=787) 4 | lastF         | Sequential         | 1.5 K 
(pid=787) 5 | modelJ        | ResNet             | 11.2 M
(pid=787) 6 | lastJ         | Sequential         | 1.5 K 
(pid=787) 7 | val_metrics   | MetricCollection   | 22.3 M
(pid=787) -----------------------------------------------------
(pid=787) 22.5 M    Trainable params
(pid=787) 0         Non-trainable params
(pid=787) 22.5 M    Total params
(pid=787) 90.136    Total estimated model params size (MB)
(pid=787) /usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:373: UserWarning: Your val_dataloader has `shuffle=True`, it is best practice 

Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]


...

Result for train_Stage1_70527_00000:
  FID: 7.26171875
  FID_cross: 15.4453125
  auroc: 0.5
  auroc_cross: 0.5188722014427185
  date: 2021-09-07_23-39-17
  done: false
  experiment_id: f4cb33f5595841f8bba99795ccc25185
  hostname: e89b1a816e20
  iterations_since_restore: 1
  loss_D: 0.1962926983833313
  loss_G: 3.5838091373443604
  node_ip: 172.28.0.2
  pid: 2634
  should_checkpoint: true
  time_since_restore: 92.48306369781494
  time_this_iter_s: 92.48306369781494
  time_total_s: 518.8621773719788
  timestamp: 1631057957
  timesteps_since_restore: 0
  training_iteration: 6
  trial_id: '70527_00000'
  


Trial name,status,loc,lr,n_fmaps,bs,loss_G,loss_D,FID,auroc,FID_cross,auroc_cross,training_iteration
train_Stage1_70527_00000,RUNNING,172.28.0.2:2634,0.0001,8,8,3.58381,0.196293,7.26172,0.5,15.4453,0.518872,6
train_Stage1_70527_00002,PAUSED,,0.000795687,16,128,9.64948,8.63587e-05,491.75,0.5,552,0.5,5
train_Stage1_70527_00003,PAUSED,,0.000636549,16,128,1.99952,1.74201,8.15625,0.480993,17.1719,0.499896,5
train_Stage1_70527_00001,PENDING,,8e-05,8,8,11.3789,1.10119e-05,80.25,0.485434,69.5,0.472704,5


Trial name,status,loc,lr,n_fmaps,bs,loss_G,loss_D,FID,auroc,FID_cross,auroc_cross,training_iteration
train_Stage1_70527_00000,RUNNING,172.28.0.2:2634,0.0001,8,8,3.58381,0.196293,7.26172,0.5,15.4453,0.518872,6
train_Stage1_70527_00002,PAUSED,,0.000795687,16,128,9.64948,8.63587e-05,491.75,0.5,552,0.5,5
train_Stage1_70527_00003,PAUSED,,0.000636549,16,128,1.99952,1.74201,8.15625,0.480993,17.1719,0.499896,5
train_Stage1_70527_00001,PENDING,,8e-05,8,8,11.3789,1.10119e-05,80.25,0.485434,69.5,0.472704,5


2021-09-07 23:39:17,832	ERROR tune.py:557 -- Trials did not complete: [train_Stage1_70527_00000, train_Stage1_70527_00001, train_Stage1_70527_00002, train_Stage1_70527_00003]
2021-09-07 23:39:17,834	INFO tune.py:561 -- Total run time: 2310.31 seconds (2309.96 seconds for the tuning loop).
2021-09-07 23:39:17,844	WARNING tune.py:566 -- Experiment has been interrupted, but the most recent state was saved. You can continue running this experiment by passing `resume=True` to `tune.run()`


Best checkpoint path found is:  /content/drive/MyDrive/Logs/F/Stage1/pbt_tanh/train_Stage1_70527_00000_0_n_fmaps=8_2021-09-07_23-00-47/checkpoint_epoch=1-step=1873/


In [ ]:
!tensorboard dev upload --logdir /content/drive/MyDrive/Logs/F/Stage1/test

In [ ]:
drive.flush_and_unmount()

## LensGAN128

In [41]:
%rm -rf drive/MyDrive/Logs/F/LensGAN128/pbt_tanh/

In [43]:
# __tune_train_checkpoint_begin
def train_LensGAN128(config, checkpoint_dir=None, num_epochs=10, num_gpus=torch.cuda.device_count()):
    # print(os.cpu_count(), torch.cuda.device_count())
    kwargs = {
        'limit_train_batches' : 0.1,
        'limit_val_batches' : 0.1,
        'progress_bar_refresh_rate' : math.ceil(8250*0.1//config['bs']),
        'max_epochs' : num_epochs,
        'prepare_data_per_node' : False,
        # If fractional GPUs passed in, convert to int.
        'gpus' : math.ceil(num_gpus),
        'logger' : TensorBoardLogger(save_dir=tune.get_trial_dir(), name='', version='.'),
        'callbacks' : [
            TuneReportCheckpointCallback(
                {
                    'loss_G': 'Stage1/G/train/loss', 
                    'loss_D': 'Stage1/D/train/loss', 
                    # Switch up the FID vlues when training on different dataset -----------------------------------------------
                    'FID': 'Stage1/val/FID_F', 
                    'FID_cross': 'Stage1/val/FID_J',
                    'auroc': 'Stage1/LensResnet(F)/val/auroc',
                    'auroc_cross': 'Stage1/LensResnet(J)/val/auroc',
                },
            ),
            ModuleDataMonitor(True),
        ],
        # 'stochastic_weight_avg' : True,
        # works with only one optimizer
        'benchmark' : True,
        'precision' : 16,
        # 'gradient_clip_val' : 0.5, 
        # 'gradient_clip_algorithm' : 'value',
    }
    
    dm = npyImageData(config, 128)                                              # Specify image width here    
    if checkpoint_dir is not None:
        kwargs['resume_from_checkpoint'] = os.path.join(checkpoint_dir, 'checkpoint')
        # model = LensGAN128.load_from_checkpoint(kwargs['resume_from_checkpoint'], config=config)
    # else:

    model = LensGAN128(config)
    trainer = pl.Trainer(**kwargs)

    trainer.fit(model, dm)
# __tune_train_checkpoint_end__

# __tune_pbt_begin__
def tune_LensGAN128_pbt(num_samples=10, num_epochs=10, gpus_per_trial=torch.cuda.device_count()):
    # print(os.cpu_count(), torch.cuda.device_count())
    analysis = tune.run(
        tune.with_parameters(
            train_LensGAN128,
            num_epochs=num_epochs,
            num_gpus=gpus_per_trial
        ),
        # Change the folder name when changing dataset--------------------------------------------------------------------------
        name='F/LensGAN128/pbt_tanh',
        metric='FID',
        mode='min',
        # stop=TrialPlateauStopper('FID'),
        resources_per_trial={'cpu': os.cpu_count(), 'gpu': gpus_per_trial},
        local_dir='./drive/MyDrive/Logs',
        # config={'lr': tune.choice([1e-4, 1e-3, 1e-5, 1e-2, 1e-6, 1e-1, 1e-7]),
        #         'bs': tune.grid_search([8, 16, 32, 64, 128]),
        #         },
        # scheduler = pbtScheduler(max_t=num_epochs, grace_period=2, reduction_factor=2),
        # Can't use RB2 as it requires mutations to be continuous
        config={'lr': 1e-4,
                'n_fmaps': tune.grid_search([8, 16, 32, 64]),
                'bs': 8,
                },
        # config = {'lr': 2.340983544823817e-05, 'n_fmaps': 32, 'bs': 8},
        scheduler = PopulationBasedTraining(time_attr='training_iteration', quantile_fraction=0.25,
                                            resample_probability=0.25,  perturbation_interval=1,
                                            hyperparam_mutations={
                                                'lr': tune.loguniform(1e-7, 1e-1),
                                                'bs': [8, 16, 32, 64, 128],
                                            },
        ),
        progress_reporter=JupyterNotebookReporter(
            overwrite=False,
            parameter_columns=['lr', 'n_fmaps', 'bs'],
            metric_columns=['loss_G', 'loss_D', 'FID', 'auroc', 'FID_cross', 
                            'auroc_cross', 'training_iteration'],
        ),
        fail_fast = True,
        # reuse_actors=True,
        num_samples=num_samples,
        resume='PROMPT',
        # restore=BEST_J_LensGAN128,
    )
    # ---------------------------------------------------------------------------------------------
    BEST_F_LensGAN128 = analysis.best_checkpoint
    print('Best checkpoint path found is: ', analysis.best_checkpoint)

# __tune_pbt_end__

if __name__ == '__main__':
    import argparse

    parser = argparse.ArgumentParser()
    parser.add_argument(
        '--smoke-test', action='store_true', help='Finish quickly for testing')
    args, _ = parser.parse_known_args()

    if args.smoke_test:
        tune_LensGAN128_pbt(num_samples=1, num_epochs=6, gpus_per_trial=torch.cuda.device_count())
    else:
        # pbt scheduler
        tune_LensGAN128_pbt(num_samples=1, num_epochs=10, gpus_per_trial=torch.cuda.device_count())

Resume from local directory? (/content/drive/MyDrive/Logs/F/LensGAN128/pbt_tanh) [y/N]: y


2021-09-08 00:53:40,708	WARNING trial_runner.py:455 -- Attempting to resume experiment from /content/drive/MyDrive/Logs/F/LensGAN128/pbt_tanh. This will ignore any new changes to the specification.
2021-09-08 00:53:40,734	INFO tune.py:484 -- TrialRunner resumed, ignoring new add_experiment.


Trial name,status,loc,lr,n_fmaps,bs,loss_G,loss_D,FID,auroc,FID_cross,auroc_cross,training_iteration
train_LensGAN128_5fa35_00003,PAUSED,,0.0001,64,8,1.65938,0.786239,22.25,0.492572,19.9219,0.49814,2
train_LensGAN128_5fa35_00000,PENDING,,0.00012,32,8,10.6631,2.57638e-05,117.688,0.501384,156.75,0.5,3
train_LensGAN128_5fa35_00002,PENDING,,0.0001,32,8,10.2784,4.06049e-05,95.125,0.509944,170.75,0.5,2
train_LensGAN128_5fa35_00001,PENDING,,0.0001,16,8,9.88384,8.79142e-05,542.5,0.4942,647.5,0.491456,2


2021-09-08 00:53:46,762	INFO trainable.py:76 -- Checkpoint size is 166470866 bytes
2021-09-08 00:53:46,905	WARNING util.py:164 -- The `start_trial` operation took 5.155 s, which may be a performance bottleneck.


Trial name,status,loc,lr,n_fmaps,bs,loss_G,loss_D,FID,auroc,FID_cross,auroc_cross,training_iteration
train_LensGAN128_5fa35_00000,RUNNING,,0.00012,32,8,10.6631,2.57638e-05,117.688,0.501384,156.75,0.5,3
train_LensGAN128_5fa35_00003,PAUSED,,0.0001,64,8,1.65938,0.786239,22.25,0.492572,19.9219,0.49814,2
train_LensGAN128_5fa35_00002,PENDING,,0.0001,32,8,10.2784,4.06049e-05,95.125,0.509944,170.75,0.5,2
train_LensGAN128_5fa35_00001,PENDING,,0.0001,16,8,9.88384,8.79142e-05,542.5,0.4942,647.5,0.491456,2


...

Result for train_LensGAN128_5fa35_00003:
  FID: 31.15625
  FID_cross: 50.875
  auroc: 0.49936342239379883
  auroc_cross: 0.513091504573822
  date: 2021-09-08_01-19-12
  done: false
  experiment_id: d70fa34cadc242c5a19ba2deaa3a4f7c
  hostname: e89b1a816e20
  iterations_since_restore: 1
  loss_D: 0.8290479779243469
  loss_G: 2.579897403717041
  node_ip: 172.28.0.2
  pid: 4862
  should_checkpoint: true
  time_since_restore: 422.9766969680786
  time_this_iter_s: 422.9766969680786
  time_total_s: 1658.0415060520172
  timestamp: 1631063952
  timesteps_since_restore: 0
  training_iteration: 4
  trial_id: 5fa35_00003
  


Trial name,status,loc,lr,n_fmaps,bs,loss_G,loss_D,FID,auroc,FID_cross,auroc_cross,training_iteration
train_LensGAN128_5fa35_00003,RUNNING,172.28.0.2:4862,0.0001,64,8,2.5799,0.829048,31.1562,0.499363,50.875,0.513092,4
train_LensGAN128_5fa35_00000,PAUSED,,0.00012,32,8,12.3994,4.88757e-06,89.1875,0.509566,109.688,0.497898,4
train_LensGAN128_5fa35_00001,PAUSED,,0.00012,64,8,10.8184,3.02044e-05,198.75,0.496637,389.75,0.495665,3
train_LensGAN128_5fa35_00002,PENDING,,0.00109864,64,16,11.4199,1.70021e-05,129.625,0.496807,163.5,0.5,3


Trial name,status,loc,lr,n_fmaps,bs,loss_G,loss_D,FID,auroc,FID_cross,auroc_cross,training_iteration
train_LensGAN128_5fa35_00003,RUNNING,172.28.0.2:4862,0.0001,64,8,2.5799,0.829048,31.1562,0.499363,50.875,0.513092,4
train_LensGAN128_5fa35_00000,PAUSED,,0.00012,32,8,12.3994,4.88757e-06,89.1875,0.509566,109.688,0.497898,4
train_LensGAN128_5fa35_00001,PAUSED,,0.00012,64,8,10.8184,3.02044e-05,198.75,0.496637,389.75,0.495665,3
train_LensGAN128_5fa35_00002,PENDING,,0.00109864,64,16,11.4199,1.70021e-05,129.625,0.496807,163.5,0.5,3


2021-09-08 01:19:13,124	ERROR tune.py:557 -- Trials did not complete: [train_LensGAN128_5fa35_00000, train_LensGAN128_5fa35_00003, train_LensGAN128_5fa35_00002, train_LensGAN128_5fa35_00001]
2021-09-08 01:19:13,137	INFO tune.py:561 -- Total run time: 1535.32 seconds (1532.01 seconds for the tuning loop).
2021-09-08 01:19:13,138	WARNING tune.py:566 -- Experiment has been interrupted, but the most recent state was saved. You can continue running this experiment by passing `resume=True` to `tune.run()`


Best checkpoint path found is:  /content/drive/MyDrive/Logs/F/LensGAN128/pbt_tanh/train_LensGAN128_5fa35_00003_3_n_fmaps=64_2021-09-08_00-09-33/checkpoint_epoch=1-step=1873/


In [44]:
drive.flush_and_unmount()

## Stage 2
Here we tune hyperparameters as we train our modified DCGAN.

In [ ]:
# __tune_train_checkpoint_begin
def train_Stage2(config, checkpoint_dir=None, num_epochs=10, num_gpus=torch.cuda.device_count()):
    # print(os.cpu_count(), torch.cuda.device_count())
    kwargs = {
        # 'limit_train_batches' : 0.05,
        # 'limit_val_batches' : 0.05,
        'progress_bar_refresh_rate' : math.ceil(8250//config['batch_size']),
        'max_epochs' : num_epochs,
        'prepare_data_per_node' : False,
        # If fractional GPUs passed in, convert to int.
        'gpus' : math.ceil(num_gpus),
        'logger' : TensorBoardLogger(save_dir=tune.get_trial_dir(), name='', version='.'),
        'callbacks' : [
            TuneReportCheckpointCallback(
                {
                    'loss_G': 'Stage2/G/train/loss', 
                    'loss_D': 'Stage2/D/train/loss', 
                    # Switch up the auroc vlues when training on different dataset -----------------------------------------------
                    'auroc': 'Stage2/ResNet(F)/val/auroc', 
                    'auroc_cross': 'Stage2/ResNet(J)/val/auroc',
                },
            ),
        ],
        # 'stochastic_weight_avg' : True,
        # works with only one optimizer
        # 'benchmark' : True,
    }
    
    dm = npyImageData(config)                                              # Specify image width here    
    if checkpoint_dir is not None:
        kwargs['resume_from_checkpoint'] = os.path.join(checkpoint_dir, 'checkpoint')
        # model = Stage2.load_from_checkpoint(kwargs['resume_from_checkpoint'], config=config)
    # else:
        # model = Stage2(config)
    model = Stage2(config)
    trainer = pl.Trainer(**kwargs)

    trainer.fit(model, dm)
# __tune_train_checkpoint_end__


# # # __tune_asha_begin__
# def tune_Stage2_asha(num_samples=10, num_epochs=10, gpus_per_trial=torch.cuda.device_count()):
#     # print(os.cpu_count(), torch.cuda.device_count())
#     analysis = tune.run(
#         tune.with_parameters(
#             train_Stage2,
#             num_epochs=num_epochs,
#             num_gpus=gpus_per_trial
#         ),
#         # Change the folder name when changing dataset--------------------------------------------------------------------------
#         name='Stage2/pbt/J',
#         metric='auroc',
#         mode='max',
#         config={'learning_rate': 1e-4,
#                 'n_fmaps': tune.grid_search([8, 16, 32, 64, 128]),
#                 'batch_size': 8,
#                 },
#         # config={'learning_rate': 0.01,
#         #         'n_fmaps': 32,
#         #         'batch_size': 32,
#         #         },
#         # stop=TrialPlateauStopper('loss_G'),
#         resources_per_trial={'cpu': os.cpu_count(),
#                              'gpu': gpus_per_trial,
#                             },
#         local_dir='./drive/MyDrive/Logs',
#         scheduler = ASHAScheduler(max_t=num_epochs, grace_period=2,  reduction_factor=2),
#         progress_reporter=JupyterNotebookReporter(
#             overwrite=True,
#             parameter_columns=['learning_rate', 'n_fmaps', 'batch_size'],
#             metric_columns=['loss_G', 'loss_D', 'auroc', 'auroc_cross', 'training_iteration'],
#             sort_by_metric=True,
#         ),
#         fail_fast = True,
#         # reuse_actors=True,
#         # num_samples=num_samples,
#         resume='PROMPT',
# #         restore='/content/drive/MyDrive/Logs/delete/train_Stage2_e42ac_00025_25_batch_size=8,learning_rate=0.01,n_fmaps=8_2021-07-28_21-16-18/checkpoint_epoch=4-step=2339',
#     )

# #     print('Best hyperparameters found were: ', analysis.best_config)

# # # __tune_asha_end__


# __tune_pbt_begin__
def tune_Stage2_pbt(num_samples=10, num_epochs=10, gpus_per_trial=torch.cuda.device_count()):
    # print(os.cpu_count(), torch.cuda.device_count())
    analysis = tune.run(
        tune.with_parameters(
            train_Stage2,
            num_epochs=num_epochs,
            num_gpus=gpus_per_trial
        ),
        # Change the folder name when changing dataset--------------------------------------------------------------------------
        name='Stage2/pbt/F',
        metric='auroc',
        mode='max',
        config={'learning_rate': 1e-4,
                'n_fmaps': tune.grid_search([8, 16, 32, 64, 128]),
                'res_depth': tune.choice([1, 2, 3, 4]),
                'batch_size': 8,
                },
        # config={'learning_rate': 0.01,
        #         'n_fmaps': 32,
        #         'batch_size': 32,
        #         },
        # stop=TrialPlateauStopper('loss_G'),
        resources_per_trial={'cpu': os.cpu_count(),
                             'gpu': gpus_per_trial,
                            },
        local_dir='./drive/MyDrive/Logs',
        scheduler = PopulationBasedTraining(time_attr='training_iteration',
                                            quantile_fraction=0.5,
                                            resample_probability=0.8,
                                            perturbation_interval=1,
                                            hyperparam_mutations={
                                                'learning_rate': tune.loguniform(1e-7, 1e-1),
                                                'batch_size': [8, 16, 32, 64, 128],
                                            },
                                            ),
        progress_reporter=JupyterNotebookReporter(
            overwrite=False,
            parameter_columns=['learning_rate', 'n_fmaps', 'res_depth', 'batch_size'],
            metric_columns=['loss_G', 'loss_D', 'auroc', 'auroc_cross', 'training_iteration'],
            sort_by_metric=True,
        ),
        fail_fast = True,
        # reuse_actors=True,
        # num_samples=num_samples,
        resume='PROMPT',
    )

    print('Best hyperparameters found were: ', analysis.best_config)

# __tune_pbt_end__


if __name__ == '__main__':
    import argparse

    parser = argparse.ArgumentParser()
    parser.add_argument(
        '--smoke-test', action='store_true', help='Finish quickly for testing')
    args, _ = parser.parse_known_args()

    if args.smoke_test:
        tune_Stage2_asha(num_samples=1, num_epochs=6, gpus_per_trial=torch.cuda.device_count())
        tune_Stage2_pbt(num_samples=1, num_epochs=6, gpus_per_trial=torch.cuda.device_count())
    else:
        # ASHA scheduler
        # tune_Stage2_asha(num_samples=1, num_epochs=10, gpus_per_trial=torch.cuda.device_count())
        # Population based training
        tune_Stage2_pbt(num_samples=1, num_epochs=30, gpus_per_trial=torch.cuda.device_count())

In [ ]:
drive.flush_and_unmount()

In [ ]:
!cat /content/drive/MyDrive/Logs/Stage2/pbt/F/train_Stage2_6f508_00000_0_n_fmaps=8_2021-08-13_14-04-54/error.txt

## StackGAN:
Here we tune hyperparameters for generating images that resemble the images from input.

In [ ]:
# __tune_train_checkpoint_begin
def train_StackGAN_tune_checkpoint(config,
                                   checkpoint_dir=None,
                                   num_epochs=10,
                                   num_gpus=torch.cuda.device_count()):
    data_dir = os.path.expanduser('/content/images/')
    trainer = pl.Trainer(
        # accumulate_grad_batches=2,
        # limit_train_batches=0.20,
        # limit_val_batches=0.20,
        num_sanity_val_steps=-1,
        max_epochs=num_epochs,
        prepare_data_per_node = False,
        # If fractional GPUs passed in, convert to int.
        gpus=math.ceil(num_gpus),
        # tpu_cores = 8,
        logger=TensorBoardLogger(save_dir=tune.get_trial_dir(), name='', version='.'),
        # progress_bar_refresh_rate=1,
        callbacks=[
            TuneReportCheckpointCallback(
                metrics={
                    'loss_G1': 'G1/train/loss/full',
                    'loss_G2': 'G2/train/loss/full',
                    'loss_D1': 'D1/train/loss',
                    'loss_D2': 'D2/train/loss',
                    'lossR': 'R/train/loss',
                    'auroc': 'Pre/val/auroc',
                },
                filename='checkpoint',
                # on='training_end'
            )
        ],
        # stochastic_weight_avg=True,
        # works with only one optimizer
        )
    dm = npyImageData(config, data_dir)
    if checkpoint_dir:
        # Currently, this leads to errors:
        # model = StackGAN.load_from_checkpoint(
        #     os.path.join(checkpoint, 'checkpoint'))
        # Workaround:
        ckpt = pl_load(
            os.path.join(checkpoint_dir, 'checkpoint'),
            map_location=lambda storage, loc: storage)
        model = StackGAN._load_model_state(
            ckpt, config=config, 
            # data_dir=data_dir
            )
        trainer.current_epoch = ckpt['epoch']
    else:
        model = StackGAN(config)

    trainer.fit(model, dm)
# __tune_train_checkpoint_end__


# __tune_asha_begin__
def tune_StackGAN_asha(num_samples=10, num_epochs=10, gpus_per_trial=torch.cuda.device_count()):
    config = {
        'learning_rate': tune.choice([1e-4]),
        'feature_maps': tune.choice([64]),
        'batch_size': tune.choice([128, 64]),
    }

    scheduler = ASHAScheduler(
        max_t=num_epochs,
        grace_period=1,
        reduction_factor=2)

    reporter = CLIReporter(
        # overwrite=True,
        parameter_columns=['learning_rate', 'feature_maps', 'batch_size'],
        metric_columns=['loss_G1', 'loss_G2', 'loss_D1', 'loss_D2', 'lossR', 'auroc', 'training_iteration'],
        )

    analysis = tune.run(
        tune.with_parameters(
            train_StackGAN_tune_checkpoint,
            num_epochs=num_epochs,
            num_gpus=gpus_per_trial),
        name='tune_StackGAN_asha_model_j',
        metric='auroc',
        mode='max',
        config=config,
        resources_per_trial={
            'cpu': os.cpu_count(),
            'gpu': gpus_per_trial,
            # 'tpu': 8,
        },
        num_samples=num_samples,
        local_dir='./drive/MyDrive/Logs',
        scheduler=scheduler,
        progress_reporter=reporter,
        # restore='/content/drive/MyDrive/Logs/tune_StackGAN_1_asha_model_j/train_StackGAN_tune_checkpoint_fa25b_00000_0_batch_size=64,feature_maps=64,learning_rate=0.0001_2021-07-06_20-23-13/checkpoint_epoch=0-step=937',
        fail_fast = True,
        resume='PROMPT',
        )

    print('Best hyperparameters found were: ', analysis.best_config)

# __tune_asha_end__


# __tune_pbt_begin__
def tune_StackGAN_pbt(num_samples=10, num_epochs=10, gpus_per_trial=torch.cuda.device_count()):
    config = {
        'learning_rate': 1e-4,
        'feature_maps': 64,
        'batch_size': 64,
    }

    scheduler = PopulationBasedTraining(
        perturbation_interval=4,
        hyperparam_mutations={
            'learning_rate': [1e-4, 1e-3],
            'feature_maps': [64, 128],
            'batch_size': [32, 64, 128]
        })

    reporter = CLIReporter(
        # overwrite=True,
        parameter_columns=['learning_rate', 'feature_maps', 'batch_size'],
        metric_columns=['loss_G1', 'loss_G2', 'loss_D1', 'loss_D2', 'lossR', 'auroc', 'training_iteration'],
        )

    analysis = tune.run(
        # resume=True,
        tune.with_parameters(
            train_StackGAN_tune_checkpoint,
            num_epochs=num_epochs,
            num_gpus=gpus_per_trial),
        name='tune_StackGAN_pbt_model_j',
        metric='auroc',
        mode='max',
        resources_per_trial={
            'cpu': os.cpu_count(),
            'gpu': gpus_per_trial,
            # 'tpu': 8,
        },
        config=config,
        num_samples=num_samples,
        scheduler=scheduler,
        progress_reporter=reporter,
        local_dir='./drive/MyDrive/Logs',
        # restore='/content/drive/MyDrive/Logs/tune_StackGAN_1_asha_model_j/train_StackGAN_tune_checkpoint_fa25b_00000_0_batch_size=64,feature_maps=64,learning_rate=0.0001_2021-07-06_20-23-13/checkpoint_epoch=0-step=937',
        fail_fast = True,
        # resume='PROMPT',
        )

    print('Best hyperparameters found were: ', analysis.best_config)

# __tune_pbt_end__


if __name__ == '__main__':
    import argparse

    parser = argparse.ArgumentParser()
    parser.add_argument(
        '--smoke-test', action='store_true', help='Finish quickly for testing')
    args, _ = parser.parse_known_args()

    if args.smoke_test:
        tune_StackGAN_asha(num_samples=1, num_epochs=6, gpus_per_trial=torch.cuda.device_count())
        tune_StackGAN_pbt(num_samples=1, num_epochs=6, gpus_per_trial=torch.cuda.device_count())
    else:
        # ASHA scheduler
        tune_StackGAN_asha(num_samples=2, num_epochs=1, gpus_per_trial=torch.cuda.device_count())
        # Population based training
        # tune_StackGAN_pbt(num_samples=8, num_epochs=5, gpus_per_trial=torch.cuda.device_count())